In [1]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [2]:
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [3]:
query_job = client.query(query)

In [4]:
us_cities = query_job.to_dataframe()

us_cities.city.value_counts().head()


Phoenix-Mesa-Scottsdale             87
Houston                             80
Los Angeles-Long Beach-Santa Ana    60
Riverside-San Bernardino-Ontario    59
San Francisco-Oakland-Fremont       58
Name: city, dtype: int64

In [5]:
query = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

query_job = client.query(query)

us_cities = query_job.to_dataframe()

us_cities.head()

,city,country
0,Houston,US
1,Houston,US
2,Houston,US
3,Houston,US
4,Houston,US


In [6]:
# For countries starting by the letter tadada...

query = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country LIKE 'F%'
        """

query_job = client.query(query)

us_cities = query_job.to_dataframe()

us_cities[:10]

,city,country
0,Reunion,FR
1,Reunion,FR
2,Reunion,FR
3,Reunion,FR
4,Reunion,FR
5,Reunion,FR
6,Reunion,FR
7,Reunion,FR
8,Reunion,FR
9,Reunion,FR


# Checking the size of the query

In [7]:
# Query to get the score column from every row where the type column has value "job"
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 2395809 bytes.


In [8]:
# Only run the query if it's less than 100 MB
Max_Query = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=Max_Query)

# Set up the query (will only run if it's less than 100 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

BadRequest: 400 GET https://dp.kaggle.net/bigquery/v2/projects/kaggle-161607/queries/ddfdedeb-ed24-4e9d-8767-046ec908b279?maxResults=0&location=US: Query exceeded limit for bytes billed: 1000000. 10485760 or higher required.